<a href="https://colab.research.google.com/github/chi-yan/notebooks/blob/master/NLP_(Bible_Text_Encoding_with_the_Universal_Text_Encoder)%2C_similarity_and_clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This Google Colab notebook uses Natural Language Processing (NLP) techniques 
to detect Bible verses that are most similar to a random block of text. Word numerical (high-ordered vectors) obtained via the "Universal Text Encoder" is used to numerically model these verses.`

Cosine similarity is to quantify similarity (in meaning) between sentences. A similiarity close to 0 indicates that two sentences are similar in meaning.

References:

http://www.ijstr.org/final-print/dec2019/Lexical-And-Semantic-Analysis-Of-Sacred-Texts-Using-Machine-Learning-And-Natural-Language-Processing.pdf (see Section 3.4)

https://tfhub.dev/google/universal-sentence-encoder/1

In [1]:
import numpy as np # linear algebra
import pandas as pd #
import tensorflow_hub as hub
import csv
import pprint
from tabulate import tabulate
from collections import OrderedDict
from operator import itemgetter
text = 'cats and dogs' #@param {type:"string", run:"auto"}
 
def cosine_similarity(A, B):
    '''
    Input:
        A: a numpy array which corresponds to a word vector
        B: A numpy array which corresponds to a word vector
    Output:
        cos: numerical number representing the cosine similarity between A and B.
    '''
    # you have to set this variable to the true label.
    cos = -10
    dot = np.dot(A, B)
    norma = np.linalg.norm(A)
    normb = np.linalg.norm(B)
    cos = dot / (norma * normb)
 
    return cos
 
if 'init' not in locals():
  init = True
  !curl https://raw.githubusercontent.com/EswarGitHub/BibleSearch/master/bible_data_set.csv -o bible_data_set.csv
  reader = csv.reader(open('bible_data_set.csv', 'r'))
  d = {}
  for row in reader:
    k,_,_,_,v = row
    d[k] = v[:-2] #    
  embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
  embeddings = embed(list(d.values()))
  versedict = {}
  for i, verse in enumerate(d):
    versedict[i] = verse
 

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 5010k  100 5010k    0     0  11.3M      0 --:--:-- --:--:-- --:--:-- 11.3M


In [2]:

def cosine_similarities(A, B): #A is the big matrix containing all embeddings, B is the sentence
    return np.divide(np.dot(A,B),np.apply_along_axis(np.linalg.norm, 1, A)*np.linalg.norm(B))[:,0]

def generateSimilarities(text, numberOfItems):
  embedded_text = embed([text])
  similarities = cosine_similarities(embeddings.numpy(), embedded_text.numpy().T)
  data = []
  for i in range(len(embeddings)):
    data.append([versedict[i],d[versedict[i]],similarities[i]])
  return(pd.DataFrame (data, columns = ['Verse', 'Text', 'Similarity']).sort_values(by=['Similarity'], ascending=False).head(numberOfItems))
#embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

In [3]:
embedded_text=embed(["Hello"])

In [4]:
similarities = cosine_similarities(embeddings.numpy(), embedded_text.numpy().T)

In [5]:
A = embeddings.numpy()
B = embedded_text.numpy().T

In [6]:
np.dot(A,B).flatten()

array([0.476579  , 0.01377824, 0.01151486, ..., 0.0474508 , 0.08687165,
       0.10028698], dtype=float32)

In [7]:
np.divide(np.dot(A,B),np.apply_along_axis(np.linalg.norm, 1, A)*np.linalg.norm(B))[:,0]

array([0.47657904, 0.01377824, 0.01151487, ..., 0.04745081, 0.08687166,
       0.10028698], dtype=float32)

In [8]:

import pickle
with open('embeddings.pickle', 'wb') as f:
    pickle.dump(embeddings, f)

In [9]:
import pickle
with open('data.pickle', 'wb') as f:
    pickle.dump(d, f)

In [10]:
embeddings

<tf.Tensor: shape=(31103, 512), dtype=float32, numpy=
array([[ 0.00468304, -0.08392566, -0.01649407, ..., -0.03823555,
         0.00346091, -0.10069176],
       [-0.05283082, -0.00314455,  0.0539399 , ..., -0.05066366,
         0.05926255,  0.007909  ],
       [-0.04331731,  0.01895817, -0.04516518, ..., -0.00918447,
         0.02742364,  0.04344477],
       ...,
       [-0.03668367, -0.02885344,  0.04155295, ..., -0.0178898 ,
         0.09710161,  0.07535172],
       [-0.01769449, -0.03582924, -0.00357681, ...,  0.01731504,
        -0.03287543,  0.01774419],
       [-0.01537722, -0.04798326, -0.03328146, ...,  0.02343452,
         0.01298577,  0.00462453]], dtype=float32)>

In [11]:
import pickle
with open('versedict.pickle', 'wb') as f:
    pickle.dump(versedict, f)

In [12]:
d['Genesis 32:15']

'Thirty milch camels with their colts, forty kine, and ten bulls, twenty she asses, and ten foals.'

In [13]:

def cosine_similarities(A, B): #A is the big matrix containing all embeddings, B is the sentence
    return np.divide(np.dot(A,B),np.apply_along_axis(np.linalg.norm, 1, A)*np.linalg.norm(B))[:,0]

In [14]:


def generateSimilarities(text, numberOfItems):
  embedded_text = embed([text])
  similarities = cosine_similarities(embeddings.numpy(), embedded_text.numpy().T)
  data = []
  for i in range(len(embeddings)):
    data.append([versedict[i],d[versedict[i]],similarities[i]])
  return(pd.DataFrame (data, columns = ['Verse', 'Text', 'Similarity']).sort_values(by=['Similarity'], ascending=False).head(numberOfItems))

pd.set_option('max_colwidth', 500)

generateSimilarities("only son",5)


,Verse,Text,Similarity
10563,1 Chronicles 7:27,"Non his son, Jehoshuah his son.",0.454879
26063,John 1:18,"No man hath seen God at any time, the only begotten Son, which is in the bosom of the Father, he hath declared him.",0.422034
564,Genesis 22:16,"And said, By myself have I sworn, saith the LORD, for because thou hast done this thing, and hast not withheld thy son, thine only son:",0.414489
25340,Luke 9:38,"And, behold, a man of the company cried out, saying, Master, I beseech thee, look upon my son: for he is mine only child.",0.399740
27396,Acts 13:33,"God hath fulfilled the same unto us their children, in that he hath raised up Jesus again; as it is also written in the second psalm, Thou art my Son, this day have I begotten thee.",0.386814


In [15]:
generateSimilarities(d['Genesis 1:1'],5)

,Verse,Text,Similarity
1,Genesis 1:1,In the beginning God created the heaven and the earth.,1.000000
35,Genesis 2:4,"These are the generations of the heavens and of the earth when they were created, in the day that the LORD God made the earth and the heavens,",0.738225
15846,Psalms 115:15,Ye are blessed of the LORD which made heaven and earth.,0.626305
16626,Proverbs 8:23,"I was set up from everlasting, from the beginning, or ever the earth was.",0.624476
15820,Psalms 113:6,"Who humbleth himself to behold the things that are in heaven, and in the earth!",0.604734


In [16]:
import tensorflow as tf


In [17]:
from sklearn.cluster import KMeans

clusters = 10

embeddings_np = embeddings.numpy()
text = pd.DataFrame(list(d.values()))
kmeans = KMeans(n_clusters=clusters, random_state=0).fit(embeddings_np)

In [18]:


def print_cluster(cluster_number):  
  locations = np.where(kmeans.labels_ == cluster_number)[0].tolist()
  text = pd.DataFrame(list(d.values()))
  print(text.iloc[locations])

In [19]:
for i in range(clusters):
  
  print_cluster(i)
  print('-'*200)


                                                                                                                                                                                       0
20                        And God said, Let the waters bring forth abundantly the moving creature that hath life, and fowl that may fly above the earth in the open firmament of heaven.
42                                                                           The name of the first is Pison: that is it which compasseth the whole land of Havilah, where there is gold;
80                  So he drove out the man; and he placed at the east of the garden of Eden Cherubims, and a flaming sword which turned every way, to keep the way of the tree of life.
104                                                                                                              If Cain shall be avenged sevenfold, truly Lamech seventy and sevenfold.
149                                                                        